In [3]:
%matplotlib notebook


In [ ]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import os
import time
from IPython.display import display, clear_output
import threading

MAX_POINTS = 5000
PICKLE_FILE = "imbalance_live.pkl"

def get_imbalance_df():
    if os.path.exists(PICKLE_FILE):
        try:
            df = pd.read_pickle(PICKLE_FILE)
            return df.tail(MAX_POINTS).copy()
        except:
            pass
    return pd.DataFrame()

# Global tracking
last_len = 0
last_ts = None
fig = None

# Setup figure once
fig = make_subplots(
    rows=5, cols=1, 
    subplot_titles=["Bid/Ask Prices", "Depth Imbalance", "Bid Flow", "Ask Flow", "Walk (ask/bid) Ratio"],
    shared_xaxes=True,
    vertical_spacing=0.03,
    row_heights=[0.26, 0.19, 0.19, 0.19, 0.17]
)

# Add traces
fig.add_trace(go.Scatter(line=dict(color="green")), row=1, col=1)
fig.add_trace(go.Scatter(line=dict(color="red")), row=1, col=1)
fig.add_trace(go.Scatter(line=dict(color="purple")), row=2, col=1)
fig.add_trace(go.Scatter(line=dict(color="blue")), row=3, col=1)
fig.add_trace(go.Scatter(line=dict(color="orange")), row=4, col=1)
fig.add_trace(go.Scatter(line=dict(color="black", width=2)), row=5, col=1)

fig.update_layout(
    height=850, 
    title="Live Order Book Monitor",
    showlegend=False,
    margin=dict(t=50, b=20, l=40, r=40)
)
fig.update_xaxes(title="Time", row=5, col=1)

def monitor():
    global last_len, last_ts
    
    while True:
        df = get_imbalance_df()
        if df.empty:
            time.sleep(0.5)
            continue
            
        df['timestamp'] = pd.to_datetime(df['timestamp'], errors='coerce')
        curr_len = len(df)
        curr_ts = df['timestamp'].iloc[-1]
        
        # Update only on new data
        if curr_len > last_len or curr_ts != last_ts:
            clear_output(wait=True)
            
            # Use FULL data AS-IS (oldest left, newest RIGHT)
            ts = df['timestamp']
            
            # Update traces with ALL data
            fig.data[0].update(x=ts, y=df['bid_price'])
            fig.data[1].update(x=ts, y=df['ask_price'])
            fig.data[2].update(x=ts, y=df['depth_imbalance'])
            fig.data[3].update(x=ts, y=df['flow_bid'])
            fig.data[4].update(x=ts, y=df['flow_ask'])
            fig.data[5].update(x=ts, y=df['ask_bid_walk'])
            
            # FULL X-axis range (all data visible)
            fig.update_xaxes(autorange=False, range=[df['timestamp'].iloc[0], df['timestamp'].iloc[-1]])
            
            # FIXED Y-AXIS RANGES (chart zoomed to these ranges, full data zoomable)
            fig.update_yaxes(range=[-10, 10], row=3, col=1, fixedrange=False)  # Bid Flow: zoomed -20 to 20
            fig.update_yaxes(range=[-10, 10], row=4, col=1, fixedrange=False)  # Ask Flow: zoomed -20 to 20
            fig.update_yaxes(range=[0, 10], row=5, col=1, fixedrange=False)    # Walk: zoomed 0 to 20
            # Rows 1-2: full auto-range (zoomable)
            fig.update_yaxes(autorange=True, row=1, col=1)
            fig.update_yaxes(autorange=True, row=2, col=1)
            
            # Print latest values
            latest = df.iloc[-1]
            print(f"🔄 {curr_len} pts | {curr_ts}")
            print(f"Bid: {latest['bid_price']:.4f}  Ask: {latest['ask_price']:.4f}")
            print(f"Depth: {latest['depth_imbalance']:.4f}  BidFlow: {latest['flow_bid']:.4f}  AskFlow: {latest['flow_ask']:.4f}")
            print(f"Walk: {latest['ask_bid_walk']:.4f}")
            print("="*60)
            
            display(fig)
            last_len, last_ts = curr_len, curr_ts
        
        time.sleep(0.5)

print("👀 Live monitoring started...")
threading.Thread(target=monitor, daemon=True).start()
display(fig)


🔄 2078 pts | 2025-12-09 12:46:58
Bid: 450.1400  Ask: 450.1800
Depth: -0.3067  BidFlow: -4.1326  AskFlow: -0.2581
Walk: 7.6451
